In [1]:
import threading
import DobotDllType as dType
#se crean definiciones para determinar errores en la comunicacion
CON_STR = {
    dType.DobotConnect.DobotConnect_NoError:  "DobotConnect_NoError",
    dType.DobotConnect.DobotConnect_NotFound: "DobotConnect_NotFound",
    dType.DobotConnect.DobotConnect_Occupied: "DobotConnect_Occupied"
    }



In [2]:
#Load Dll and get the CDLL object
api = dType.load()
#Connect Dobot
state = dType.ConnectDobot(api, "", 115200)
print("datos de retorno de conexion ",[state])
print("Id Robot ",[state[1]])
print("Connect status:",CON_STR[state[0]])


python： ('64bit', 'WindowsPE')
datos de retorno de conexion  [[0, 2, 0, 'DobotSerial', '0.0.0', 1, 0, 0.0]]
Id Robot  [2]
Connect status: DobotConnect_NoError


In [3]:

#para realizar movimientos en x,y,z se define el valor de la posicion al que se desea que se mueva el robot
dType.SetPTPCmd(api, dType.PTPMode.PTPMOVLXYZMode, 200 , -20, 30, -90, isQueued = 1)[0]
dType.SetQueuedCmdStartExec(api)
print(dType.GetHOMEParams(api))
print(dType.GetKinematics(api))
dType.SetQueuedCmdStopExec(api)

[180.0, 0.0, 0.0, 0.0]
[0.0, 0.0]


In [4]:
#para realizar movimientos de cada articulacion se define el valor del angulo al que se desea que rote dicha articulacion
dType.SetPTPCmd(api, dType.PTPMode.PTPMOVJANGLEMode, 15 , 25, 25,0, isQueued = 1)[0]
dType.SetQueuedCmdStartExec(api)
print(dType.GetHOMEParams(api))
print(dType.GetKinematics(api))
dType.SetQueuedCmdStopExec(api)

[180.0, 0.0, 0.0, 0.0]
[0.0, 0.0]


In [24]:
dType.SetEndEffectorSuctionCupEx(api,True,False) #True par aactivar ventosa el segundo paramttro, y para desactivar cambiarlo por False

In [38]:

if (state[0] == dType.DobotConnect.DobotConnect_NoError):
    #Clean Command Queued
    dType.SetQueuedCmdClear(api)
    
    
    #Async Motion Params Setting
    #definir la posicion home
    #dType.SetHOMEParams(api, 200, 0, 130, 0, isQueued = 1)
    #definir la velocidad y aceleracion de cada Joint
    #dType.SetPTPJointParams(api, 200, 200, 200, 200, 200, 200, 200, 200, isQueued = 1)
    #definir una velovidad y aceleracion global de movimiento
    dType.SetPTPCommonParams(api, 80, 20, isQueued = 1)

    #Async Home mover a posicion home
    #dType.SetHOMECmd(api, temp = 0, isQueued = 1)

    #Async PTP Motion
    lastIndex1 = dType.SetPTPCmd(api, dType.PTPMode.PTPMOVLXYZMode, 200 , 0, 130, 0, isQueued = 1)[0]
    lastIndex2 = dType.SetPTPCmd(api, dType.PTPMode.PTPMOVLXYZMode, 220 , 50, -41, -90, isQueued = 1)[0]
    lastIndex3 = dType.SetPTPCmd(api, dType.PTPMode.PTPMOVLXYZMode, 180 , -50, 30, -90, isQueued = 1)[0]
    index=[lastIndex1, lastIndex2, lastIndex3]

    #Start to Execute Command Queue
    dType.SetQueuedCmdStartExec(api)
    st = False
    #Wait for Executing Last Command 
    for i in range(3):
        while index[i] > dType.GetQueuedCmdCurrentIndex(api)[0]:
            dType.dSleep(100)
        dType.SetEndEffectorSuctionCupEx(api,st,st)   #la variable st cambia a true y false para activar y desactivar la ventosa en cada punto        
        print("move doIt")
        print(st)
        st = not st

    #Stop to Execute Command Queued
    dType.SetQueuedCmdStopExec(api)



move doIt
False
move doIt
True
move doIt
False


In [4]:

#Disconnect Dobot
dType.DisconnectDobot(api)